In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=239, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

325 documents
40 classes ['aula_apoteker', 'aula_farmasi', 'aula_tesla', 'bem', 'dimana_aula', 'fsi', 'gazebo', 'himabio', 'himafarma', 'himafi', 'himakom', 'himamia', 'himasta', 'himatika', 'inventaris_labbiologi', 'inventaris_labfarmasi', 'inventaris_labfisika', 'inventaris_labkimia', 'inventaris_labkom', 'jawab_ya', 'lab_anfis', 'lab_bio_dasar', 'lab_geo_fisika', 'lab_instrumen', 'lab_kimia', 'lab_mikrobiologi', 'labkom', 'msc', 'musholla', 'rk_gd1_lt1', 'rk_gd1_lt2', 'ruang_apoteker', 'sapaan', 'sgo', 'sudah_selesai', 'tanya_inventaris', 'tanya_ruangan', 'terimakasih', 'toilet', 'uptkm']
118 unique lemmatized words ['1.1', '1.2', '2.1', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '3.1', '3.2', '3.3', 'ada', 'afternoon', 'alat', 'alat-alat', 'anfis', 'apa', 'apoteker', 'aula', 'bai', 'banar', 'banyak', 'bem', 'benar', 'betul', 'bio', 'biologi', 'bro', 'bujur', 'bye', 'byee', 'cari', 'dadah', 'dah', 'dan', 'dasar', 'di', 'dimana', 'ekologi', 'farmasi', 'fisika', 'fsi', 'gazebo', 'geo',

C:\Users\ridho\AppData\Local\Temp\ipykernel_14436\1969829324.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\ridho\anaconda3\envs\chatbot_fakultas\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/239
65/65 [==============================] - 4s 7ms/step - loss: 3.6402 - accuracy: 0.0677
Epoch 2/239
65/65 [==============================] - 0s 6ms/step - loss: 3.4514 - accuracy: 0.1108
Epoch 3/239
65/65 [==============================] - 0s 7ms/step - loss: 3.1968 - accuracy: 0.1446
Epoch 4/239
65/65 [==============================] - 0s 5ms/step - loss: 3.0653 - accuracy: 0.1815
Epoch 5/239
65/65 [==============================] - 0s 6ms/step - loss: 2.7666 - accuracy: 0.2585
Epoch 6/239
65/65 [==============================] - 0s 6ms/step - loss: 2.6365 - accuracy: 0.2769
Epoch 7/239
65/65 [==============================] - 0s 7ms/step - loss: 2.4603 - accuracy: 0.3108
Epoch 8/239
65/65 [==============================] - 0s 6ms/step - loss: 2.2818 - accuracy: 0.3415
Epoch 9/239
65/65 [==============================] - 0s 6ms/step - loss: 2.1367 - accuracy: 0.3785
Epoch 10/239
65/65 [==============================] - 0s 6ms/step - loss: 1.9010 - accuracy: 0.4246
Epoch 11/